In [2]:
#csv먼저 읽어오기
import numpy as np
import matplotlib.pyplot as plt
import tensorflow as tf
import pandas as pd
from tensorflow.keras.models import Model
from tensorflow.keras.layers import Flatten,Dense
from tensorflow.keras.layers import Input
from tensorflow.keras.optimizers import Adam
from sklearn.preprocessing import MinMaxScaler
from sklearn.model_selection import train_test_split

from sklearn.metrics import classification_report
from tensorflow.keras.utils import plot_model
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense
from tensorflow.keras.utils import plot_model


In [15]:
import os

image_dir = 'data/CIFAR.train/cifar-10/train/train'

# 확장자가 .png인 파일 개수 세기 (CIFAR-10은 보통 PNG 형식)
image_files = [f for f in os.listdir(image_dir) if f.endswith('.png')]
print(f"이미지 개수: {len(image_files)}")


이미지 개수: 50000


In [ ]:
import os
import shutil
import pandas as pd
from sklearn.model_selection import train_test_split

# CSV 파일 경로
csv_path = 'data/CIFAR.train/cifar-10/trainLabels.csv'
# 기본 폴더 생성
base_dir = 'cifar10_full'
os.makedirs(base_dir)

# 이미지 디렉토리 경로
image_dir = 'data/CIFAR.train/cifar-10/train'

# CSV 파일 읽기
df = pd.read_csv(csv_path)

# 이미지 파일 경로 및 라벨
image_paths = [os.path.join(image_dir, f"{id}.png") for id in df['id']]  # CIFAR-10 이미지 파일은 .png 확장자
labels = df['label']

# 클래스 이름 목록 (0부터 9까지)
class_names = ['airplane', 'automobile', 'bird', 'cat', 'deer', 'dog', 'frog', 'horse', 'ship', 'truck']

# 데이터셋을 train(80%)과 validation(20%)로 분할
train_paths, val_paths, train_labels, val_labels = train_test_split(image_paths, labels, test_size=0.2, random_state=42)



train_dir = os.path.join(base_dir, 'train')
os.makedirs(train_dir)
validation_dir = os.path.join(base_dir, 'validation')
os.makedirs(validation_dir)

# 각 클래스 폴더 생성
for class_name in class_names:
    os.makedirs(os.path.join(train_dir, class_name))
    os.makedirs(os.path.join(validation_dir, class_name))

# 이미지를 각 폴더에 복사하는 함수
def copy_images(paths, labels, target_dir):
    for path, label in zip(paths, labels):
        # 라벨에 해당하는 클래스 폴더 경로 설정
        class_name = class_names[label]
        class_dir = os.path.join(target_dir, class_name)
        
        # 이미지 복사
        shutil.copy(path, class_dir)

# train, validation 폴더에 이미지 복사
copy_images(train_paths, train_labels, train_dir)
copy_images(val_paths, val_labels, validation_dir)

print("이미지 복사 완료")


In [2]:
import os
import shutil
import pandas as pd
from sklearn.model_selection import train_test_split

# 경로 설정
csv_path = 'data/CIFAR.train/cifar-10/trainLabels.csv'
image_dir = 'data/CIFAR.train/cifar-10/train/train'
base_dir = 'cifar10_full'
train_dir = os.path.join(base_dir, 'train')
validation_dir = os.path.join(base_dir, 'validation')

# 클래스 이름
class_names = ['airplane', 'automobile', 'bird', 'cat', 'deer',
               'dog', 'frog', 'horse', 'ship', 'truck']

# CSV 읽기
df = pd.read_csv(csv_path)

# 파일 이름에 확장자 붙이기 (.png라 가정)
df['filename'] = df['id'].astype(str) + '.png'

# 클래스별로 이미지 나누고 복사
for class_name in class_names:
    # 해당 클래스의 파일만 선택
    class_df = df[df['label'] == class_name]

    # 파일 분할 (train: 4,000 / validation: 1,000)
    train_files, val_files = train_test_split(class_df['filename'], test_size=0.2, random_state=42)

    # 이미지 복사
    for fname in train_files:
        src = os.path.join(image_dir, fname)
        dst = os.path.join(train_dir, class_name, fname)
        shutil.copyfile(src, dst)

    for fname in val_files:
        src = os.path.join(image_dir, fname)
        dst = os.path.join(validation_dir, class_name, fname)
        shutil.copyfile(src, dst)

print("이미지 복사 완료: train 4,0000장 / validation 1,0000장 per class")


이미지 복사 완료: train 4,0000장 / validation 1,0000장 per class


In [3]:
import optuna
import warnings
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.applications.efficientnet import preprocess_input
from tensorflow.keras import mixed_precision#16비트를 통해서 계산이 빨라짐 
mixed_precision.set_global_policy('mixed_float16')
warnings.filterwarnings('ignore')


train_dir='cifar10_full/train'
validation_dir='cifar10_full/validation'

train_datagen = ImageDataGenerator(
    preprocessing_function=preprocess_input,
    rotation_range=30,
    width_shift_range=0.1,
    height_shift_range=0.1,
    shear_range=0.2,
    zoom_range=0.2,
    fill_mode='nearest'  # 오타 수정
)

validation_datagetn=ImageDataGenerator(preprocessing_function=preprocess_input)

train_generator=train_datagen.flow_from_directory(train_dir,
                                                 classes=['airplane', 'automobile', 'bird', 'cat', 'deer',
                                                           'dog', 'frog', 'horse', 'ship', 'truck'],
                                                 target_size=(380,380),
                                                 batch_size=40,
                                                 class_mode='categorical')

validation_generator=validation_datagetn.flow_from_directory(validation_dir,
                                                 classes=['airplane', 'automobile', 'bird', 'cat', 'deer',
                                                           'dog', 'frog', 'horse', 'ship', 'truck'],
                                                 target_size=(380,380),
                                                 batch_size=40,
                                                 class_mode='categorical')



Found 40000 images belonging to 10 classes.
Found 10000 images belonging to 10 classes.


In [6]:
# import os
# import tensorflow as tf
# from tensorflow.keras.applications.efficientnet import preprocess_input
# from tensorflow.data import AUTOTUNE
# import numpy as np

# # 데이터셋 디렉토리 경로
# train_dir = 'cifar10_full/train'
# validation_dir = 'cifar10_full/validation'

# # 레이블 매핑 (디렉토리 구조에 맞게 설정)
# class_names = ['airplane', 'automobile', 'bird', 'cat', 'deer', 'dog', 'frog', 'horse', 'ship', 'truck']

# # 이미지를 로드하고 전처리하는 함수
# def process_image(image_path):
#     image = tf.io.read_file(image_path)  # 이미지 파일 읽기
#     image = tf.image.decode_jpeg(image, channels=3)  # 이미지 디코딩
#     image = tf.image.resize(image, (380, 380))  # 크기 변경
#     image = preprocess_input(image)  # EfficientNet 전처리
#     return image

# # 레이블을 0~9 범위의 정수로 변환하는 함수
# def get_label(file_path):
#     parts = tf.strings.split(file_path, os.sep)
#     class_name = parts[-2]  # 파일 경로에서 클래스 이름 추출
#     label = tf.argmax(class_name == class_names)  # 클래스 이름을 인덱스로 변환
#     return label

# # 이미지 파일 경로를 Tensor로 가져오는 함수
# def get_image_paths_and_labels(directory):
#     image_paths = []
#     labels = []
#     for class_name in class_names:
#         class_dir = os.path.join(directory, class_name)
#         for file_name in os.listdir(class_dir):
#             image_paths.append(os.path.join(class_dir, file_name))
#             labels.append(class_name)
#     return image_paths, labels

# # tf.data.Dataset을 사용하여 데이터셋을 생성
# def create_dataset(image_paths, labels, batch_size, shuffle=True):
#     dataset = tf.data.Dataset.from_tensor_slices((image_paths, labels))

#     # 전처리: 이미지 처리, 정수 레이블 처리
#     def load_and_process(path, label_str):
#         image = process_image(path)
#         label = get_label(tf.convert_to_tensor(path))  # label_str 대신 path에서 추출
#         return image, label

#     dataset = dataset.map(load_and_process, num_parallel_calls=AUTOTUNE)

#     if shuffle:
#         dataset = dataset.shuffle(buffer_size=len(image_paths))
#     dataset = dataset.batch(batch_size)
#     dataset = dataset.prefetch(AUTOTUNE)
#     return dataset


# # 훈련 및 검증 데이터셋 생성
# train_paths, train_labels = get_image_paths_and_labels(train_dir)
# validation_paths, validation_labels = get_image_paths_and_labels(validation_dir)

# batch_size = 8  # 배치 크기 설정

# train_dataset = create_dataset(train_paths, train_labels, batch_size, shuffle=True)
# validation_dataset = create_dataset(validation_paths, validation_labels, batch_size, shuffle=False)


In [7]:
# import tensorflow as tf
# from tensorflow.keras.models import Model
# from tensorflow.keras.layers import GlobalAveragePooling2D, Dense, Dropout,BatchNormalization,Activation
# from tensorflow.keras.optimizers import Adam
# from tensorflow.keras.applications import EfficientNetB4

# # EfficientNetB4 모델 불러오기 (이미 학습된 모델 사용, weights='imagenet' 옵션)
# base_model = EfficientNetB4(weights='imagenet', 
#                             include_top=False, 
#                             input_shape=(380, 380, 3))

# # 기계학습을 위한 네트워크 구축
# base_model.trainable = False  # 먼저 EfficientNetB4 모델을 동결시켜서 학습하지 않게 설정

# # 모델의 출력층 추가
# x = base_model.output
# x = GlobalAveragePooling2D()(x)
# x= Dense(units=64)(x)
# x=BatchNormalization()(x)
# x=Activation('relu')(x)
# # 공간 정보를 1차원으로 축소
# x = Dropout(0.2)(x)  # Dropout 추가 (과적합 방지)
# x = Dense(64, activation='relu')(x)  # Dense 층 추가
# output = Dense(units=10, activation='softmax')(x)  # 최종 클래스 예측 층

# # 전체 모델 구성
# model = Model(inputs=base_model.input, outputs=output)

# # 모델 요약
# model.summary()

# # 모델 컴파일
# model.compile(optimizer=Adam(learning_rate=1e-4),
#               loss='sparse_categorical_crossentropy',
#               metrics=['accuracy'])


Model: "model_2"
__________________________________________________________________________________________________
 Layer (type)                Output Shape                 Param #   Connected to                  
 input_5 (InputLayer)        [(None, 380, 380, 3)]        0         []                            
                                                                                                  
 rescaling_8 (Rescaling)     (None, 380, 380, 3)          0         ['input_5[0][0]']             
                                                                                                  
 normalization_4 (Normaliza  (None, 380, 380, 3)          7         ['rescaling_8[0][0]']         
 tion)                                                                                            
                                                                                                  
 rescaling_9 (Rescaling)     (None, 380, 380, 3)          0         ['normalization_4[0][0]'

In [ ]:
# # 모델 학습


# history = model.fit(
#     train_dataset,
#     steps_per_epoch=len(train_dataset),
#     epochs=10, 
#     validation_data=validation_dataset, 
#     validation_steps=len(validation_dataset),# 검증용 데이터셋
#     verbose=1
# )


Epoch 1/10


In [4]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import GlobalAveragePooling2D, Dense, BatchNormalization, Activation, Dropout
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.callbacks import EarlyStopping, ModelCheckpoint
from tensorflow.keras.applications import EfficientNetB4
from tensorflow.keras import mixed_precision

# mixed precision 설정
mixed_precision.set_global_policy('mixed_float16')

# EfficientNetB4 불러오기
pretrained_network = EfficientNetB4(weights='imagenet',
                                    include_top=False,
                                    input_shape=(380, 380, 3))

# 모든 레이어 동결
for layer in pretrained_network.layers:
    layer.trainable = False  

# 모델 구성
model = Sequential()
model.add(pretrained_network)
model.add(GlobalAveragePooling2D())
model.add(Dense(units=64))
model.add(BatchNormalization())
model.add(Activation('relu'))
model.add(Dropout(rate=0.2))
model.add(Dense(units=10, activation='softmax'))  

# 모델 컴파일
model.compile(optimizer=Adam(learning_rate=1e-4),   
              loss='categorical_crossentropy',      
              metrics=['accuracy'])

# 콜백 정의
call = EarlyStopping(patience=5, monitor='val_loss', verbose=1)
check = ModelCheckpoint(filepath='./eva.weights.h5',
                        save_best_only=True, 
                        save_weights_only=True,
                        monitor='val_accuracy',
                        verbose=1)


In [6]:
history = model.fit(
    train_generator,
    steps_per_epoch=len(train_generator),   # 전체 샘플 수 / batch_size 와 동일
    epochs=30,
    validation_data=validation_generator,
    validation_steps=len(validation_generator),
    verbose=1,
    callbacks=[call, check]
)


Epoch 1/30
1000/1000 [==============================] - ETA: 0s - loss: 1.0053 - accuracy: 0.6482   
Epoch 1: val_accuracy improved from 0.72030 to 0.72800, saving model to ./eva.weights.h5
1000/1000 [==============================] - 567s 567ms/step - loss: 1.0053 - accuracy: 0.6482 - val_loss: 0.8005 - val_accuracy: 0.7280
Epoch 2/30
1000/1000 [==============================] - ETA: 0s - loss: 0.9988 - accuracy: 0.6507   
Epoch 2: val_accuracy did not improve from 0.72800
1000/1000 [==============================] - 561s 561ms/step - loss: 0.9988 - accuracy: 0.6507 - val_loss: 0.7952 - val_accuracy: 0.7253
Epoch 3/30
1000/1000 [==============================] - ETA: 0s - loss: 0.9848 - accuracy: 0.6568   
Epoch 3: val_accuracy improved from 0.72800 to 0.73160, saving model to ./eva.weights.h5
1000/1000 [==============================] - 562s 562ms/step - loss: 0.9848 - accuracy: 0.6568 - val_loss: 0.7881 - val_accuracy: 0.7316
Epoch 4/30
1000/1000 [==============================] - E

In [7]:
model.save('geun-les.model.h5')

In [19]:
# # EfficientNet 일부만 fine-tuning
# for layer in pretrained_network.layers:
#     layer.trainable = True

# for layer in pretrained_network.layers[:-30]:  # 마지막 30개만 학습
#     layer.trainable = False

# # 다시 컴파일 (learning rate 줄이는 것도 잘하셨습니다)
# model.compile(optimizer=Adam(learning_rate=(1e-3) * 0.1), 
#               loss='categorical_crossentropy',  
#               metrics=['accuracy'])

# # 학습
# history = model.fit(
#     train_generator,
#     steps_per_epoch=len(train_generator),
#     epochs=10,
#     validation_data=validation_generator,
#     validation_steps=len(validation_generator),
#     verbose=1,
#     callbacks=[call, check]

# )
# model.save('final_model.h5')  # 학습 완료된 모델을 'final_model.h5'로 저장
# print("Model saved as 'final_model.h5'")

Epoch 1/10
1000/1000 [==============================] - ETA: 0s - loss: 0.9615 - accuracy: 0.6654     
Epoch 1: val_accuracy improved from 0.72600 to 0.79040, saving model to ./eva.weights.h5
1000/1000 [==============================] - 682s 668ms/step - loss: 0.9615 - accuracy: 0.6654 - val_loss: 0.6094 - val_accuracy: 0.7904
Epoch 2/10
1000/1000 [==============================] - ETA: 0s - loss: 0.7646 - accuracy: 0.7347   
Epoch 2: val_accuracy improved from 0.79040 to 0.82120, saving model to ./eva.weights.h5
1000/1000 [==============================] - 696s 696ms/step - loss: 0.7646 - accuracy: 0.7347 - val_loss: 0.5125 - val_accuracy: 0.8212
Epoch 3/10
1000/1000 [==============================] - ETA: 0s - loss: 0.6892 - accuracy: 0.7610   
Epoch 3: val_accuracy improved from 0.82120 to 0.84110, saving model to ./eva.weights.h5
1000/1000 [==============================] - 641s 639ms/step - loss: 0.6892 - accuracy: 0.7610 - val_loss: 0.4705 - val_accuracy: 0.8411
Epoch 4/10
1000/1

In [6]:

model = tf.keras.models.load_model('final_model.h5')
model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])
model.fit(
    train_generator,
    steps_per_epoch=len(train_generator),
    epochs=10,
    validation_data=validation_generator,
    validation_steps=len(validation_generator),
    verbose=1,
    callbacks=[call, check]

)


Epoch 1/10
1000/1000 [==============================] - ETA: 0s - loss: 0.1311 - accuracy: 0.7553     
Epoch 1: val_accuracy improved from -inf to 0.83690, saving model to ./eva.weights.h5
1000/1000 [==============================] - 577s 566ms/step - loss: 0.1311 - accuracy: 0.7553 - val_loss: 0.0816 - val_accuracy: 0.8369
Epoch 2/10
1000/1000 [==============================] - ETA: 0s - loss: 0.1040 - accuracy: 0.7926   
Epoch 2: val_accuracy improved from 0.83690 to 0.84900, saving model to ./eva.weights.h5
1000/1000 [==============================] - 567s 567ms/step - loss: 0.1040 - accuracy: 0.7926 - val_loss: 0.0784 - val_accuracy: 0.8490
Epoch 3/10
1000/1000 [==============================] - ETA: 0s - loss: 0.0956 - accuracy: 0.8111   
Epoch 3: val_accuracy improved from 0.84900 to 0.86360, saving model to ./eva.weights.h5
1000/1000 [==============================] - 567s 567ms/step - loss: 0.0956 - accuracy: 0.8111 - val_loss: 0.0700 - val_accuracy: 0.8636
Epoch 4/10
1000/1000

In [ ]:
import os
import numpy as np
import pandas as pd
from tensorflow.keras.preprocessing.image import load_img, img_to_array
from tensorflow.keras.applications.efficientnet import preprocess_input
from tensorflow.keras.models import load_model

# 모델 불러오기
model = load_model('geun-les.model.h5')  

# 이미지 크기
IMG_SIZE = (380, 380)  # EfficientNet의 입력 크기에 맞춰 380x380으로 설정

# 클래스 이름 목록 (CIFAR-10의 경우 10개의 클래스)
class_names = ['airplane', 'automobile', 'bird', 'cat', 'deer', 'dog', 'frog', 'horse', 'ship', 'truck']
index_to_class = {i: class_name for i, class_name in enumerate(class_names)}  

# CIFAR-10 test 이미지 경로
test_dir = 'data/CIFAR.train/cifar-10/test/test'

# CIFAR-10 test 폴더 내 이미지 파일 리스트
test_files = [f for f in sorted(os.listdir(test_dir)) if f.lower().endswith(('jpg', 'png'))]  # 이미지 파일만 선택

# 예측 결과를 저장할 리스트
ids, labels = [], []

# 각 테스트 이미지에 대해 예측
for fname in test_files:
    img_path = os.path.join(test_dir, fname)
    
    try:
        # 이미지 열기 (380x380 크기로 맞추기)
        img = load_img(img_path, target_size=IMG_SIZE)
        img_array = img_to_array(img)
        img_array = preprocess_input(img_array)  # EfficientNet에 맞게 전처리
        img_array = np.expand_dims(img_array, axis=0)  # 배치 차원 추가

        # 예측
        pred = model.predict(img_array, verbose=0)
        class_idx = np.argmax(pred, axis=1)[0]  # 가장 확률 높은 클래스
        class_label = index_to_class[class_idx]

        # 결과 저장
        file_id = int(os.path.splitext(fname)[0])  # 파일 이름에서 id를 추출 (예: '3.jpg' -> 3)
        ids.append(file_id)
        labels.append(class_label)
    
    except Exception as e:
        print(f"Error loading image {fname}: {e}")
        continue  # 오류가 발생한 이미지는 건너뛰기

# 결과를 DataFrame에 담기
df = pd.DataFrame({'id': ids, 'label': labels})

# 'id'로 정렬
df = df.sort_values('id')  

# CSV로 저장
df.to_csv('submissiond.csv', index=False)

print("Prediction completed and saved to 'submissiond.csv'")
